# Robot Demo

This demo proves **motors** work via **direct motor control** (`/ros_robot_controller/set_motor_speeds`).

- Movement (forward/back/turn/strafe/diagonal)
- Drift (wheel-level)
- Horn (meep meep)

Plus:
- Eyes (RGB) via `eyes_lib` (no fast-sdk)

Camera + TTS are separate and can stay as-is for now.

In [ ]:
# --- Bootstrap: add common/lib and lessons/lib to sys.path ---
from pathlib import Path
import sys

def find_repo_root() -> Path:
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'common').is_dir() and (p / 'lessons').is_dir():
            return p
    raise FileNotFoundError('Could not find repo root (needs common/ and lessons/)')

ROOT = find_repo_root()
COMMON_LIB = ROOT / 'common' / 'lib'
LESSONS_LIB = ROOT / 'lessons' / 'lib'

for path in (COMMON_LIB, LESSONS_LIB):
    if str(path) not in sys.path:
        sys.path.insert(0, str(path))

print('Repo root:', ROOT)
print('Using common lib:', COMMON_LIB)
print('Using lessons lib:', LESSONS_LIB)
print('Python:', sys.version.split()[0])

In [ ]:
import time, importlib
import robot_moves as rm
importlib.reload(rm)

bot = rm.RobotMoves(base_speed=220, rate_hz=30)
print('RobotMoves ready. BASE_SPEED=', rm.BASE_SPEED, 'RATE_HZ=', rm.RATE_HZ)

In [ ]:
# --- Eyes setup (no fast-sdk) ---
# Uses common/lib/eyes_lib.py
import eyes_lib
importlib.reload(eyes_lib)

eyes = None
try:
    eyes = eyes_lib.get_eyes()
    print('Eyes ready:', eyes)
    
    # Optional helpers if your eyes implementation exposes them
    if hasattr(eyes, 'diagnose'):
        print('diagnose():', eyes.diagnose())
except Exception as e:
    print('Eyes setup failed:', e)
    eyes = None

In [ ]:
# --- Eyes demo ---
# This is safe: it only runs if eyes initialised.
if not eyes:
    print('Skipping eyes demo (eyes not available).')
else:
    def set_eye(i, r, g, b):
        # prefer set_rgb if it exists
        if hasattr(eyes, 'set_rgb'):
            return eyes.set_rgb(i, r, g, b)
        # fallback: some implementations use set(index, r, g, b)
        if hasattr(eyes, 'set'):
            return eyes.set(i, r, g, b)
        raise AttributeError('Eyes object has no set_rgb() or set()')

    print('Eyes: RGB cycle')
    set_eye(0, 255, 0, 0); time.sleep(0.4)
    set_eye(0, 0, 255, 0); time.sleep(0.4)
    set_eye(0, 0, 0, 255); time.sleep(0.4)
    set_eye(0, 0, 0, 0)

    # Optional scan if your eyes impl supports it
    if hasattr(eyes, 'scan_indices'):
        print('scan_indices():', eyes.scan_indices())

    print('Eyes demo done')

In [ ]:
# Quick safety: stop first
bot.stop()
time.sleep(0.2)
print('Stopped.')

In [ ]:
print('Forward / Backward')
bot.forward(0.6)
time.sleep(0.2)
bot.backward(0.6)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Turn left / right')
bot.turn_left(0.5)
time.sleep(0.2)
bot.turn_right(0.5)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Strafe left / right')
bot.left(0.6)
time.sleep(0.2)
bot.right(0.6)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Diagonals')
bot.diagonal_left(0.6)
time.sleep(0.2)
bot.diagonal_right(0.6)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Drift demo (wheel-level, no /cmd_vel)')
bot.drift_left(seconds=0.9, turn_blend=0.55)
time.sleep(0.2)
bot.drift_right(seconds=0.9, turn_blend=0.55)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Horn: meep meep')
bot.horn(block=True)
print('Done')